In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [2]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('Terry_stops.csv')
data.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,Reported Date,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,Asian,Male,2015-10-16T00:00:00,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,-,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,-,-,2015-03-19T00:00:00,07:59:00,-,-,-,NaN,N,N,-,-,-
2,-,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,White,Male,2015-03-21T00:00:00,19:12:00,-,-,-,NaN,N,-,-,-,-
3,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,-,-,2015-04-01T00:00:00,04:55:00,-,-,-,NaN,N,N,-,-,-
4,-,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,Black or African American,Male,2015-04-03T00:00:00,00:41:00,-,-,-,NaN,N,N,-,-,-


## Initial Inspection of Data

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48525 entries, 0 to 48524
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         48525 non-null  object
 1   Subject ID                48525 non-null  int64 
 2   GO / SC Num               48525 non-null  int64 
 3   Terry Stop ID             48525 non-null  int64 
 4   Stop Resolution           48525 non-null  object
 5   Weapon Type               48525 non-null  object
 6   Officer ID                48525 non-null  object
 7   Officer YOB               48525 non-null  int64 
 8   Officer Gender            48525 non-null  object
 9   Officer Race              48525 non-null  object
 10  Subject Perceived Race    48525 non-null  object
 11  Subject Perceived Gender  48525 non-null  object
 12  Reported Date             48525 non-null  object
 13  Reported Time             48525 non-null  object
 14  Initial Call Type     

In [4]:
# removing unnecessary columns from dataframe
df = data.drop(columns=['Subject ID', 'GO / SC Num', 'Terry Stop ID', 'Reported Time'], axis=1)

In [5]:
# Inspection of Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48525 entries, 0 to 48524
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         48525 non-null  object
 1   Stop Resolution           48525 non-null  object
 2   Weapon Type               48525 non-null  object
 3   Officer ID                48525 non-null  object
 4   Officer YOB               48525 non-null  int64 
 5   Officer Gender            48525 non-null  object
 6   Officer Race              48525 non-null  object
 7   Subject Perceived Race    48525 non-null  object
 8   Subject Perceived Gender  48525 non-null  object
 9   Reported Date             48525 non-null  object
 10  Initial Call Type         48525 non-null  object
 11  Final Call Type           48525 non-null  object
 12  Call Type                 48525 non-null  object
 13  Officer Squad             47919 non-null  object
 14  Arrest Flag           

In [6]:
df.describe()

,Officer YOB
count,48525.000000
mean,1983.091355
std,9.083316
min,1900.000000
25%,1978.000000
50%,1985.000000
75%,1990.000000
max,1998.000000


In [7]:
#reformat col names 
df.rename(columns={'Subject Age Group':'subject_age_group', 'Stop Resolution':'stop_resolution', 'Weapon Type':'weapon_type', 'Officer ID':'officer_id',
       'Officer YOB':'officer_yob', 'Officer Gender':'officer_gender', 'Officer Race':'officer_race',
       'Subject Perceived Race':'subject_perceived_race', 'Subject Perceived Gender':'subject_perceived_gender', 'Reported Date':'reported_date',
       'Initial Call Type':'initial_call_type', 'Final Call Type':'final_call_type', 'Call Type':'call_type', 'Officer Squad': 'officer_squad',
       'Arrest Flag':'arrest_flag', 'Frisk Flag':'frisk_flag', 'Precinct':'precinct', 'Sector':'sector', 'Beat':'beat'}, inplace=True)

In [8]:
# Inspecting value counts for each column
for col in df.columns:
    try:
        print(df[col].value_counts())
    except:
        print(df[col].value_counts())   
    print('\n')

26 - 35         16171
36 - 45         10345
18 - 25          9676
46 - 55          6276
56 and Above     2469
1 - 17           1991
-                1597
Name: subject_age_group, dtype: int64


Field Contact               19716
Offense Report              15860
Arrest                      12041
Referred for Prosecution      728
Citation / Infraction         180
Name: stop_resolution, dtype: int64


None                                    32565
-                                       13060
Lethal Cutting Instrument                1482
Knife/Cutting/Stabbing Instrument         707
Handgun                                   300
Firearm Other                             100
Blunt Object/Striking Implement            93
Club, Blackjack, Brass Knuckles            49
Firearm                                    44
Mace/Pepper Spray                          31
Other Firearm                              27
Firearm (unk type)                         15
Taser/Stun Gun                             10


## Data Cleaning Episode 1: The NaN Menace

I begin the data cleaning process by inspecting columns and their NaN's to determine how to handle them. Some were converted to the 'None' value that was already present in the column values. Others were left in because they would not be used in dataframe used for modelling.

In [9]:
# Some Columns have an '-' so will convert to NaN
#  and then inspect for NaN Values

df.replace('-', np.nan, inplace=True)
df.isna().sum()

subject_age_group            1597
stop_resolution                 0
weapon_type                 13060
officer_id                     24
officer_yob                     0
officer_gender                  0
officer_race                    0
subject_perceived_race       1851
subject_perceived_gender      235
reported_date                   0
initial_call_type           13234
final_call_type             13234
call_type                   13234
officer_squad                 606
arrest_flag                     0
frisk_flag                    478
precinct                     9930
sector                      10141
beat                        10075
dtype: int64

In [10]:
# Subject Age Group (10 year increments) as reported by the officer.

df['subject_age_group'].value_counts(dropna=False)

26 - 35         16171
36 - 45         10345
18 - 25          9676
46 - 55          6276
56 and Above     2469
1 - 17           1991
NaN              1597
Name: subject_age_group, dtype: int64

In [11]:
# Convert NaN to 'Not Specified'
df['subject_age_group'] = df['subject_age_group'].replace(np.nan,'Not Specified')

In [12]:
# Type of weapon, if any, identified during a search or frisk of the subject. Indicates "None" if no weapons was found.

df['weapon_type'].value_counts(dropna=False)

None                                    32565
NaN                                     13060
Lethal Cutting Instrument                1482
Knife/Cutting/Stabbing Instrument         707
Handgun                                   300
Firearm Other                             100
Blunt Object/Striking Implement            93
Club, Blackjack, Brass Knuckles            49
Firearm                                    44
Mace/Pepper Spray                          31
Other Firearm                              27
Firearm (unk type)                         15
Taser/Stun Gun                             10
None/Not Applicable                         9
Club                                        9
Fire/Incendiary Device                      8
Rifle                                       7
Shotgun                                     3
Automatic Handgun                           2
Personal Weapons (hands, feet, etc.)        2
Brass Knuckles                              1
Blackjack                         

In [13]:
# Converting NaN's to 'None' for weapon type column
df['weapon_type'] = df['weapon_type'].replace(np.nan,'None')

# Convert 'None/Not Applicable' to 'None' for alignment
df['weapon_type'] = df['weapon_type'].replace('None/Not Applicable','None')

In [14]:
# Key identifying unique officers in the dataset.

df['officer_id'].value_counts(dropna=False)

7456      430
7634      347
7773      333
7765      315
7758      313
         ... 
8543        1
5445        1
6022        1
6421        1
6363        1
Name: officer_id, Length: 1225, dtype: int64

In [15]:
# Perceived race of the subject, as reported by the officer.

df['subject_perceived_race'].value_counts(dropna=False)

White                                        23716
Black or African American                    14465
Unknown                                       2793
NaN                                           1851
Hispanic                                      1684
Asian                                         1594
American Indian or Alaska Native              1392
Multi-Racial                                   809
Other                                          152
Native Hawaiian or Other Pacific Islander       69
Name: subject_perceived_race, dtype: int64

In [16]:
# Convert NaN's to Unknown so that they are aligned

df['subject_perceived_race'] = df['subject_perceived_race'].replace(np.nan,'Unknown')

In [17]:
# Perceived gender of the subject, as reported by the officer.

df['subject_perceived_gender'].value_counts(dropna=False)

Male                                                         38119
Female                                                        9801
Unable to Determine                                            326
NaN                                                            235
Unknown                                                         37
Gender Diverse (gender non-conforming and/or transgender)        7
Name: subject_perceived_gender, dtype: int64

In [18]:
#convert NaN's and 'Unable to determine' to 'Unknown' so that they are aligned

df['subject_perceived_gender'] = df['subject_perceived_gender'].replace(np.nan,'Unknown')
df['subject_perceived_gender'] = df['subject_perceived_gender'].replace('Unable to Determine','Unknown')

In [19]:
# Initial classification of the call as assigned by 911

df['initial_call_type'].value_counts(dropna=False)

NaN                                               13234
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW         3234
SUSPICIOUS PERSON, VEHICLE OR INCIDENT             3098
DISTURBANCE, MISCELLANEOUS/OTHER                   2506
ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)     2108
                                                  ...  
PHONE - OBSCENE OR NUISANCE PHONE CALLS               1
ESCAPE - PRISONER                                     1
MISSING - (ALZHEIMER, ENDANGERED, ELDERLY)            1
MISSING - ADULT                                       1
REQUEST TO WATCH                                      1
Name: initial_call_type, Length: 168, dtype: int64

In [20]:
# Final classification of the call as assigned by the primary officer closing the event.

df['final_call_type'].value_counts(dropna=False)

NaN                                                13234
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON            3920
--PROWLER - TRESPASS                                3406
--DISTURBANCE - OTHER                               2802
--ASSAULTS, OTHER                                   2386
                                                   ...  
FIGHT - VERBAL/ORAL (NO WEAPONS)                       1
MVC - UNK INJURIES                                     1
NOISE - DIST, GENERAL (CONST, RESID, BALL PLAY)        1
BIAS -RACIAL, POLITICAL, SEXUAL MOTIVATION             1
WARRANT PICKUP - FROM OTHER AGENCY                     1
Name: final_call_type, Length: 211, dtype: int64

In [21]:
# How the call was received by the communication center.

df['call_type'].value_counts(dropna=False)

911                              22127
NaN                              13234
ONVIEW                            9364
TELEPHONE OTHER, NOT 911          3429
ALARM CALL (NOT POLICE ALARM)      361
TEXT MESSAGE                         9
SCHEDULED EVENT (RECURRING)          1
Name: call_type, dtype: int64

In [22]:
# Functional squad assignment (not budget) of the officer as reported by the Data Analytics Platform (DAP).

df['officer_squad'].value_counts(dropna=False)

TRAINING - FIELD TRAINING SQUAD                   5210
WEST PCT 1ST W - DAVID/MARY                       1583
WEST PCT 2ND W - D/M RELIEF                       1041
SOUTHWEST PCT 2ND W - FRANK                       1007
WEST PCT 1ST W - KING/QUEEN                        899
                                                  ... 
BURG/THEFT/JUV - NORTH                               1
BURG/THEFT/JUV - WEST                                1
COMM - INTERNET AND TELEPHONE REPORTING (ITRU)       1
TRAF - MOTORCYCLE UNIT - T3 SQUAD                    1
TRAINING - ADVANCED - SQUAD C                        1
Name: officer_squad, Length: 174, dtype: int64

In [23]:
# Indicator of whether a "frisk" was conducted, by the officer, of the subject, during the Terry Stop.
df['frisk_flag'].value_counts(dropna=False)

N      37056
Y      10991
NaN      478
Name: frisk_flag, dtype: int64

In [24]:
# Dropping NaN's from frisk flag column bc it only represents .9% of the data set and convert column to object type
df = df.dropna(how='any',subset=['frisk_flag'])
df['frisk_flag'] = df['frisk_flag'].astype('object')

In [25]:
# Precinct of the address associated with the underlying Computer Aided Dispatch (CAD) event. 
# Not necessarily where the Terry Stop occurred.

df['precinct'].value_counts(dropna=False)

West         11821
North        10559
NaN           9793
East          6346
South         5754
Southwest     2293
SouthWest     1227
Unknown        196
OOJ             37
FK ERROR        21
Name: precinct, dtype: int64

In [26]:
# Sector of the address associated with the underlying Computer Aided Dispatch (CAD) event. 
# Not necessarily where the Terry Stop occurred.

df['sector'].value_counts(dropna=False)

NaN       10000
E          2313
M          2234
N          2156
K          1889
K          1734
B          1642
L          1606
D          1498
R          1438
M          1365
F          1363
S          1328
U          1285
D          1249
O          1136
J          1107
G          1077
C          1028
E           993
Q           954
W           929
Q           894
N           768
F           755
R           684
O           655
B           601
S           514
G           499
U           489
L           482
W           471
C           435
J           423
99           53
Name: sector, dtype: int64

In [27]:
# Beat of the address associated with the underlying Computer Aided Dispatch (CAD) event. 
# Not necessarily where the Terry Stop occurred.

df['beat'].value_counts(dropna=False)

NaN       9934
N3        1154
E2        1080
K3        1058
M2         842
          ... 
N1          86
99          53
99          35
OOJ         25
S            2
Name: beat, Length: 107, dtype: int64

## Data Cleaning Episode 2: The Attack of the Converts

The next part of the data cleaning involves converting the values in columns, the data type of the columns, and the removal of any outliers in the data. I also add a month and year column using the reported date column. Finally, I convert the arrest flag frisk flag columns from Y:N to 1:0. 

In [28]:
# Convert Officer YOB to Age

df['officer_yob'] = 2021 - df['officer_yob'] 

In [29]:
# rename officer_yob column to officer age

df.rename(columns={'officer_yob':'officer_age'}, inplace=True)

In [30]:
# Inspecting age outliers

df.nlargest(50,['officer_age'])

,subject_age_group,stop_resolution,weapon_type,officer_id,officer_age,officer_gender,officer_race,subject_perceived_race,subject_perceived_gender,reported_date,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat
1048,Not Specified,Field Contact,None,NaN,121,N,Unknown,Unknown,Unknown,2019-07-27T00:00:00,NaN,NaN,NaN,NaN,N,Y,West,K,K3
1049,Not Specified,Field Contact,None,NaN,121,N,Unknown,Unknown,Unknown,2019-07-27T00:00:00,NaN,NaN,NaN,NaN,N,Y,West,K,K3
1050,Not Specified,Field Contact,None,NaN,121,N,Unknown,Unknown,Unknown,2019-07-27T00:00:00,NaN,NaN,NaN,NaN,N,Y,West,K,K3
1069,Not Specified,Field Contact,None,NaN,121,N,Unknown,White,Male,2019-09-10T00:00:00,NaN,NaN,NaN,NaN,N,N,East,E,E3
3634,18 - 25,Field Contact,None,-9,121,N,Unknown,Black or African American,Male,2015-05-19T00:00:00,NaN,NaN,NaN,NaN,N,N,NaN,NaN,NaN
7259,18 - 25,Field Contact,None,-9,121,N,Unknown,Unknown,Female,2017-06-06T00:00:00,NaN,NaN,NaN,NaN,N,N,NaN,NaN,NaN
9291,18 - 25,Arrest,None,-9,121,N,Unknown,White,Male,2018-02-07T00:00:00,NaN,NaN,NaN,NaN,N,N,West,Q,Q1
11351,18 - 25,Field Contact,None,IV70,121,M,Unknown,Unknown,Male,2019-05-30T00:00:00,NaN,NaN,NaN,NaN,N,N,East,E,E1
11912,18 - 25,Field Contact,None,NaN,121,N,Unknown,White,Female,2019-11-27T00:00:00,NaN,NaN,NaN,NaN,N,N,West,M,M3
12551,18 - 25,Offense Report,None,NaN,121,N,Unknown,American Indian or Alaska Native,Female,2019-10-29T00:00:00,NaN,NaN,NaN,NaN,N,N,West,M,M3


In [31]:
# dropping rows where officer age is 121 bc there is insufficient data in those rows 

df.drop((df[df['officer_age'] == 121]).index, inplace = True)

In [32]:
# Convert reported_date column to datetime type

df['reported_date'] = pd.to_datetime(df['reported_date'], yearfirst=True)

In [33]:
# add columns for year and month from 'reported_date' column
df['year'] = df['reported_date'].dt.year
df['month'] = df['reported_date'].dt.month
df['year'] = df['year'].astype('object')
df['month'] = df['month'].astype('object')

In [34]:
# Convert Arrest and Frisk columns to 1's and 0's
df['arrest_flag'] = df['arrest_flag'].replace({'Y':1, 'N':0})
df['frisk_flag'] = df['frisk_flag'].replace({'Y':1, 'N':0})

## Data Cleaning Episode 3: The Revenge of the CSV

The final part of the data cleaning process is to inspect the arrest flag column which will be our target and the the final dataframe which will be saved as a csv.

In [35]:
# inspecting value counts for arrest flag column
df['arrest_flag'].value_counts(dropna=False)

0    44182
1     3830
Name: arrest_flag, dtype: int64

In [36]:
# Final Inspection of Dataframe
df.shape

(48012, 21)

In [37]:
df.head()

,subject_age_group,stop_resolution,weapon_type,officer_id,officer_age,officer_gender,officer_race,subject_perceived_race,subject_perceived_gender,reported_date,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat,year,month
0,Not Specified,Arrest,None,7500,37,M,Black or African American,Asian,Male,2015-10-16,NaN,NaN,NaN,SOUTH PCT 1ST W - ROBERT,0,0,South,O,O2,2015,10
1,Not Specified,Field Contact,None,5670,56,M,White,Unknown,Unknown,2015-03-19,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2015,3
3,Not Specified,Field Contact,None,7539,58,M,White,Unknown,Unknown,2015-04-01,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2015,4
4,Not Specified,Field Contact,None,6973,44,M,White,Black or African American,Male,2015-04-03,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2015,4
5,Not Specified,Field Contact,None,7402,48,M,White,Black or African American,Male,2015-04-05,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2015,4


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48012 entries, 0 to 48524
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   subject_age_group         48012 non-null  object        
 1   stop_resolution           48012 non-null  object        
 2   weapon_type               48012 non-null  object        
 3   officer_id                48012 non-null  object        
 4   officer_age               48012 non-null  int64         
 5   officer_gender            48012 non-null  object        
 6   officer_race              48012 non-null  object        
 7   subject_perceived_race    48012 non-null  object        
 8   subject_perceived_gender  48012 non-null  object        
 9   reported_date             48012 non-null  datetime64[ns]
 10  initial_call_type         35004 non-null  object        
 11  final_call_type           35004 non-null  object        
 12  call_type         

In [39]:
# save cleaned dataframe as a csv
df.to_csv('Terry_stops_cleaned.csv', index=False)